In [1]:
import deepchem
deepchem.__version__

2023-02-22 00:09:58.264134: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


'2.7.1'

In [2]:
import os
import numpy as np
import pandas as pd

import tempfile

from rdkit import Chem
from rdkit.Chem import AllChem
import deepchem as dc

from deepchem.utils import download_url, load_from_disk

In [3]:
data_dir = os.path.join(os.getenv('HOME'), 'datasets')
dataset_file = os.path.join(data_dir, "pdbbind_core_df.csv.gz")

if not os.path.exists(dataset_file):
    print('File does not exist. Downloading file...')
    download_url("https://s3-us-west-1.amazonaws.com/deepchem.io/datasets/pdbbind_core_df.csv.gz")
    print('File downloaded...')

raw_dataset = load_from_disk(dataset_file)
raw_dataset = raw_dataset[['pdb_id', 'smiles', 'label']]

In [4]:
raw_dataset.head(2)

,pdb_id,smiles,label
0,2d3u,CC1CCCCC1S(O)(O)NC1CC(C2CCC(CN)CC2)SC1C(O)O,6.92
1,3cyx,CC(C)(C)NC(O)C1CC2CCCCC2C[NH+]1CC(O)C(CC1CCCCC...,8.00


In [5]:
from simtk.openmm.app import PDBFile
from pdbfixer import PDBFixer

from deepchem.utils.vina_utils import prepare_inputs

In [6]:
# consider one protein-ligand complex for visualization
pdbid = raw_dataset['pdb_id'].iloc[1]
ligand = raw_dataset['smiles'].iloc[1]

In [7]:
%%time
fixer = PDBFixer(pdbid=pdbid)
PDBFile.writeFile(fixer.topology, fixer.positions, open('%s.pdb' % (pdbid), 'w'))

p, m = None, None
# fix protein, optimize ligand geometry, and sanitize molecules
try:
    p, m = prepare_inputs('%s.pdb' % (pdbid), ligand)
except:
    print('%s failed PDB fixing' % (pdbid)) 

if p and m:  # protein and molecule are readable by RDKit
    print(pdbid, p.GetNumAtoms())
    Chem.rdmolfiles.MolToPDBFile(p, '%s.pdb' % (pdbid))
    Chem.rdmolfiles.MolToPDBFile(m, 'ligand_%s.pdb' % (pdbid))

<timed exec>:7: DeprecationWarning: Call to deprecated function prepare_inputs. Please use the corresponding function in deepchem.utils.docking_utils.
[00:10:04] WARNING: not removing hydrogen atom without neighbors
[00:10:04] WARNING: not removing hydrogen atom without neighbors


3cyx 1512
CPU times: user 1.36 s, sys: 104 ms, total: 1.47 s
Wall time: 3.52 s


In [8]:
import mdtraj as md
import nglview

from IPython.display import display, Image

In [9]:
protein_mdtraj = md.load_pdb('3cyx.pdb')
ligand_mdtraj = md.load_pdb('ligand_3cyx.pdb')

In [10]:
v = nglview.show_mdtraj(ligand_mdtraj)

In [11]:
display(v)

NGLWidget()

In [12]:
view = nglview.show_mdtraj(protein_mdtraj)
display(view)

NGLWidget()

In [13]:
finder = dc.dock.binding_pocket.ConvexHullPocketFinder()
pockets = finder.find_pockets('3cyx.pdb')
len(pockets)

36

In [14]:
vpg = dc.dock.pose_generation.VinaPoseGenerator()

In [15]:
!mkdir -p vina_test

In [17]:
%%time
complexes, scores = vpg.generate_poses(molecular_complex=('3cyx.pdb', 'ligand_3cyx.pdb'),  # protein-ligand files for docking,
                                       out_dir='vina_test',
                                       generate_scores=True
                                      )

Computing Vina grid ... done.
Performing docking (random seed: -280968138) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************
CPU times: user 16min 39s, sys: 2.71 s, total: 16min 42s
Wall time: 3min 15s


/home/kiddos/.local/lib/python3.8/site-packages/vina/vina.py:260: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._voxels = np.ceil(np.array(box_size) / self._spacing).astype(np.int)


In [18]:
scores

[-9.447, -9.167, -9.026, -8.903, -8.711, -8.67, -8.639, -8.628, -8.595]

In [19]:
complex_mol = Chem.CombineMols(complexes[0][0], complexes[0][1])

In [20]:
v = nglview.show_rdkit(complex_mol)
display(v)

NGLWidget()

In [21]:
docker = dc.dock.docking.Docker(pose_generator=vpg)
posed_complex, score = next(docker.dock(molecular_complex=('3cyx.pdb', 'ligand_3cyx.pdb'),
                                          use_pose_generator_scores=True))


mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1       -9.447          0          0
   2       -9.167      1.948      4.348
   3       -9.026      1.665      3.963
   4       -8.903      1.567      2.517
   5       -8.711      1.264      3.554
   6        -8.67      4.514      8.007
   7       -8.639      2.147      4.019
   8       -8.628      2.982      11.23
   9       -8.595      3.595      7.753
Computing Vina grid ... done.
Performing docking (random seed: 921347504) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************


In [26]:
pdbids = raw_dataset['pdb_id'].values
ligand_smiles = raw_dataset['smiles'].values

In [31]:
if not os.path.exists('pdbs'):
    os.mkdir('pdbs')


for (pdbid, ligand) in zip(pdbids, ligand_smiles):
    fixer = PDBFixer(url='https://files.rcsb.org/download/%s.pdb' % (pdbid))
    PDBFile.writeFile(fixer.topology, fixer.positions, open('pdbs/%s.pdb' % (pdbid), 'w'))

    p, m = None, None
    # skip pdb fixing for speed
    try:
        p, m = prepare_inputs('pdbs/%s.pdb' % (pdbid), ligand, replace_nonstandard_residues=False,
                              remove_heterogens=False, remove_water=False,
                              add_hydrogens=False)
    except:
        print('%s failed sanitization' % (pdbid)) 

    if p and m:  # protein and molecule are readable by RDKit
        Chem.rdmolfiles.MolToPDBFile(p, 'pdbs/%s.pdb' % (pdbid))
        Chem.rdmolfiles.MolToPDBFile(m, 'pdbs/ligand_%s.pdb' % (pdbid))

/tmp/ipykernel_2921672/3781450640.py:12: DeprecationWarning: Call to deprecated function prepare_inputs. Please use the corresponding function in deepchem.utils.docking_utils.
  p, m = prepare_inputs('pdbs/%s.pdb' % (pdbid), ligand, replace_nonstandard_residues=False,
[00:26:56] UFFTYPER: Unrecognized atom type: S_5+4 (7)
/tmp/ipykernel_2921672/3781450640.py:12: DeprecationWarning: Call to deprecated function prepare_inputs. Please use the corresponding function in deepchem.utils.docking_utils.
  p, m = prepare_inputs('pdbs/%s.pdb' % (pdbid), ligand, replace_nonstandard_residues=False,
/tmp/ipykernel_2921672/3781450640.py:12: DeprecationWarning: Call to deprecated function prepare_inputs. Please use the corresponding function in deepchem.utils.docking_utils.
  p, m = prepare_inputs('pdbs/%s.pdb' % (pdbid), ligand, replace_nonstandard_residues=False,
/tmp/ipykernel_2921672/3781450640.py:12: DeprecationWarning: Call to deprecated function prepare_inputs. Please use the corresponding func

1hfs failed sanitization


/tmp/ipykernel_2921672/3781450640.py:12: DeprecationWarning: Call to deprecated function prepare_inputs. Please use the corresponding function in deepchem.utils.docking_utils.
  p, m = prepare_inputs('pdbs/%s.pdb' % (pdbid), ligand, replace_nonstandard_residues=False,
/tmp/ipykernel_2921672/3781450640.py:12: DeprecationWarning: Call to deprecated function prepare_inputs. Please use the corresponding function in deepchem.utils.docking_utils.
  p, m = prepare_inputs('pdbs/%s.pdb' % (pdbid), ligand, replace_nonstandard_residues=False,
/tmp/ipykernel_2921672/3781450640.py:12: DeprecationWarning: Call to deprecated function prepare_inputs. Please use the corresponding function in deepchem.utils.docking_utils.
  p, m = prepare_inputs('pdbs/%s.pdb' % (pdbid), ligand, replace_nonstandard_residues=False,
/tmp/ipykernel_2921672/3781450640.py:12: DeprecationWarning: Call to deprecated function prepare_inputs. Please use the corresponding function in deepchem.utils.docking_utils.
  p, m = prepare_

In [32]:
proteins = [f for f in os.listdir('pdbs') if len(f) == 8 and f.endswith('.pdb')]
ligands = [f for f in os.listdir('pdbs') if f.startswith('ligand') and f.endswith('.pdb')]

In [33]:
failures = set([f[:-4] for f in proteins]) - set([f[7:-4] for f in ligands])
for pdbid in failures:
    proteins.remove(pdbid + '.pdb')
    
len(proteins), len(ligands)

(190, 190)

In [34]:
pdbids = [f[:-4] for f in proteins]
small_dataset = raw_dataset[raw_dataset['pdb_id'].isin(pdbids)]
labels = small_dataset.label

In [36]:
fp_featurizer = dc.feat.CircularFingerprint(size=2048)

features = fp_featurizer.featurize([Chem.MolFromPDBFile('pdbs/' + l) for l in ligands])

dataset = dc.data.NumpyDataset(X=features, y=labels, ids=pdbids)
train_dataset, test_dataset = dc.splits.RandomSplitter().train_test_split(dataset, seed=42)

In [37]:
from sklearn.ensemble import RandomForestRegressor

from deepchem.utils.evaluate import Evaluator
import pandas as pd
     

seed = 66 # Set a random seed to get stable results
sklearn_model = RandomForestRegressor(n_estimators=100, max_features='sqrt')
sklearn_model.random_state = seed
model = dc.models.SklearnModel(sklearn_model)
model.fit(train_dataset)

/home/kiddos/.local/lib/python3.8/site-packages/sklearn/utils/optimize.py:18: DeprecationWarning: Please use `line_search_wolfe2` from the `scipy.optimize` namespace, the `scipy.optimize.linesearch` namespace is deprecated.
  from scipy.optimize.linesearch import line_search_wolfe2, line_search_wolfe1
/home/kiddos/.local/lib/python3.8/site-packages/sklearn/utils/optimize.py:18: DeprecationWarning: Please use `line_search_wolfe1` from the `scipy.optimize` namespace, the `scipy.optimize.linesearch` namespace is deprecated.
  from scipy.optimize.linesearch import line_search_wolfe2, line_search_wolfe1


In [38]:
# use Pearson correlation so metrics are > 0
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)

evaluator = Evaluator(model, train_dataset, [])
train_r2score = evaluator.compute_model_performance([metric])
print("RF Train set R^2 %f" % (train_r2score["pearson_r2_score"]))

evaluator = Evaluator(model, test_dataset, [])
test_r2score = evaluator.compute_model_performance([metric])
print("RF Test set R^2 %f" % (test_r2score["pearson_r2_score"]))

RF Train set R^2 0.941112
RF Test set R^2 0.004135


In [39]:
# Compare predicted and true values
list(zip(model.predict(train_dataset), train_dataset.y))[:5]

[(7.053399999999994, 7.4),
 (6.637393333333341, 6.85),
 (4.64911333333333, 3.4),
 (6.844500000000011, 6.72),
 (10.147633333333324, 11.06)]

In [40]:
list(zip(model.predict(test_dataset), test_dataset.y))[:5]

[(5.9431249999999975, 4.21),
 (6.4083000000000006, 8.7),
 (5.680599999999998, 6.39),
 (5.703366666666662, 4.94),
 (6.4501333333333335, 9.21)]

In [44]:
fp_featurizer = dc.feat.ContactCircularFingerprint(size=2048)

ligand_paths = ['pdbs/' + l for l in ligands]
protein_paths = ['pdbs/' + p for p in proteins]
features = fp_featurizer.featurize(zip(ligand_paths, protein_paths))
dataset = dc.data.NumpyDataset(X=features, y=labels, ids=pdbids)
train_dataset, test_dataset = dc.splits.RandomSplitter().train_test_split(dataset, seed=42)

[00:36:15] Explicit valence for atom # 1205 O, 3, is greater than permitted
[00:36:25] Explicit valence for atom # 2136 O, 3, is greater than permitted
[00:36:29] Explicit valence for atom # 1822 C, 5, is greater than permitted
[00:36:37] Explicit valence for atom # 2197 C, 8, is greater than permitted
[00:36:37] Explicit valence for atom # 29 C, 5, is greater than permitted
[00:36:50] Explicit valence for atom # 8 C, 5, is greater than permitted
[00:37:00] Explicit valence for atom # 1849 O, 3, is greater than permitted
[00:37:10] Explicit valence for atom # 282 O, 3, is greater than permitted
[00:37:19] Explicit valence for atom # 214 O, 3, is greater than permitted
[00:37:27] Explicit valence for atom # 2030 C, 5, is greater than permitted
[00:37:34] Explicit valence for atom # 11 C, 5, is greater than permitted
[00:37:38] Explicit valence for atom # 1 C, 5, is greater than permitted
[00:37:44] Explicit valence for atom # 1064 O, 3, is greater than permitted
[00:37:47] Explicit vale

In [45]:
seed = 42 # Set a random seed to get stable results
sklearn_model = RandomForestRegressor(n_estimators=100, max_features='sqrt')
sklearn_model.random_state = seed
model = dc.models.SklearnModel(sklearn_model)
model.fit(train_dataset)

In [46]:
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)

evaluator = Evaluator(model, train_dataset, [])
train_r2score = evaluator.compute_model_performance([metric])
print("RF Train set R^2 %f" % (train_r2score["pearson_r2_score"]))

evaluator = Evaluator(model, test_dataset, [])
test_r2score = evaluator.compute_model_performance([metric])
print("RF Test set R^2 %f" % (test_r2score["pearson_r2_score"]))

RF Train set R^2 0.358572
RF Test set R^2 0.000020
